In [102]:
import pandas as pd
import pyodbc
import psycopg2
from sqlalchemy import create_engine,text
import sys
import os
import sys
import os

root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))  # Adjust based on your project structure
sys.path.append(root_dir)
from script.sentiment_analysis import add_sentiment, aggregate_sentiment
from script.thematic_analysis import assign_themes
from script.db_util import *
from script.insert_to_db import *
print(f"Added {root_dir} to sys.path")
# Debug: Print current working directory and Python path
print('Current working directory:', os.getcwd())
print('Python path:', sys.path)

Added e:\KAIM\Customer-Experience-Analytics-for-Fintech-Apps to sys.path
Current working directory: e:\KAIM\Customer-Experience-Analytics-for-Fintech-Apps\notebooks
Python path: ['C:\\Users\\Naod.Amare\\AppData\\Local\\Programs\\Python\\Python313\\python313.zip', 'C:\\Users\\Naod.Amare\\AppData\\Local\\Programs\\Python\\Python313\\DLLs', 'C:\\Users\\Naod.Amare\\AppData\\Local\\Programs\\Python\\Python313\\Lib', 'C:\\Users\\Naod.Amare\\AppData\\Local\\Programs\\Python\\Python313', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fintech-Apps\\venv', '', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fintech-Apps\\venv\\Lib\\site-packages', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fintech-Apps\\venv\\Lib\\site-packages\\win32', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fintech-Apps\\venv\\Lib\\site-packages\\win32\\lib', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fintech-Apps\\venv\\Lib\\site-packages\\Pythonwin', 'e:\\KAIM', 'e:\\KAIM', 'e:\\KAIM\\Customer-Experience-Analytics-for-Fi

In [100]:
# Query to check if the 'reviews' table exists in the 'public' schema
result = pd.read_sql("SELECT EXISTS (SELECT FROM information_schema.tables WHERE table_schema = 'public' AND table_name = 'reviews')", engine)
print(result.head())

   exists
0    True


In [32]:
# Load data
file_path = os.path.join('..', 'data', 'clean_reviews.csv')
df = pd.read_csv(file_path)
df.head()

,user_name,rating,date,review,review_id,bank,source
0,Daniel Ephrem,5,2025-06-09 18:31:56,So bad now and hard to use,0,CBE,Google Play
1,abdulkerim habib,5,2025-06-09 16:20:06,"it is so amazing app. but, it is better to upd...",1,CBE,Google Play
2,Abdulhalim Bedre,4,2025-06-09 11:49:09,v.good app,2,CBE,Google Play
3,Moha Yimer,1,2025-06-09 01:24:23,very good app,3,CBE,Google Play
4,Puoch chuol Wath,5,2025-06-08 21:52:23,Very amazing app indeed. I'm enjoying it,4,CBE,Google Play


In [33]:
df = add_sentiment(df)
df[["review", "sentiment_label", "sentiment_score"]].head()

,review,sentiment_label,sentiment_score
0,So bad now and hard to use,negative,0.999806
1,"it is so amazing app. but, it is better to upd...",positive,0.949643
2,v.good app,positive,0.995270
3,very good app,positive,0.999868
4,Very amazing app indeed. I'm enjoying it,positive,0.999882


In [36]:
agg = aggregate_sentiment(df)
agg.head()


,bank,rating,sentiment_score
0,BOA,1,0.977978
1,BOA,2,0.935626
2,BOA,3,0.961892
3,BOA,4,0.958368
4,BOA,5,0.958823


In [37]:
themes = assign_themes(df)
themes

e:\KAIM\Customer-Experience-Analytics-for-Fintech-Apps\venv\Lib\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ll', 've'] not in stop_words.
  warnings.warn(


{'Other': ['good',
  'app',
  'good app',
  'nice',
  'work',
  'ok',
  'wow',
  'like',
  'bank',
  'amazing',
  'fast',
  'use',
  'great',
  'easy',
  'banking',
  'bad',
  'excellent',
  'application',
  'dashen',
  'super']}

#### Load the Data to csv

In [38]:
df.to_csv("../data/reviews_with_sentiment.csv", index=False)
agg.to_csv("../data/sentiment_aggregate.csv", index=False)

#### dump banksdata to banks table

In [54]:
# Assuming 'df' is your original DataFrame
pg_bank=write_unique_banks_to_postgres(df, engine)


#### dump review data to reviews table

In [95]:
pg_reviews= insert_reviews(df,engine )

#### Validate the reviews if its load to the DB

In [97]:
# Query to check if the 'reviews' table exists in the 'public' schema
result = pd.read_sql("SELECT * from reviews", engine)
print(result.head())

  name                                             review  rating  \
0  CBE                         So bad now and hard to use       5   
1  CBE  it is so amazing app. but, it is better to upd...       5   
2  CBE                                         v.good app       4   
3  CBE                                      very good app       1   
4  CBE           Very amazing app indeed. I'm enjoying it       5   

           review_date       source sentiment_label  sentiment_score  
0  2025-06-09 18:31:56  Google Play        negative         0.999806  
1  2025-06-09 16:20:06  Google Play        positive         0.949643  
2  2025-06-09 11:49:09  Google Play        positive         0.995270  
3  2025-06-09 01:24:23  Google Play        positive         0.999868  
4  2025-06-08 21:52:23  Google Play        positive         0.999882  
